# 3. Image similarity metrics
Intensity-based image registration techniques calculate the transformation between two images based solely on the image intensities (i.e. selection or determination of corresponding point pairs is not needed as in point-based registration). These techniques rely on image similarity metrics that measure the similarity between two images based on their intensities. Intensity-based image registration usually works by iteratively optimizing such a similarity metric.

In this section, you will implement two such image similarity metrics: correlation and mutual information. The computation of the mutual information between two images relies on their joint histogram, so one of the exercises deals with the implementation of this intermediate step. In the project work section you will use the similarity metrics to find the optimal rotation transformation that aligns two images.

Before you start, load your favorite test image in Python. You're going to use this image to test your implementation. Some of the examples below assume you work with images of type `uint8`, i.e. pixel intensities in the $[0, 255]$ range, but the provided functions are equipped to work with arbitrary image types.

## 3.1. Normalized cross-correlation
The normalized cross-correlation between two images $I$ and $J$ with pixels $i$ and respective
mean intensities $\overline{I}$ and $\overline{J}$ is:

\begin{equation}
C=\frac{\sum_{i=1}^{n}(I(i)-\overline{I})(J(i)-\overline{J})}{\sqrt{\sum_{i=1}^{n}(I(i)-\overline{I})^{2} \sum_{j=1}^{n}(J(i)-\overline{J})^{2}}}
\end{equation}

where n is the number of image pixels. If we reshape the 2D images to vectors (in Python this can be done with `numpy.reshape()`), the expression for the normalized cross-correlation can be rewritten using vector multiplication operators (which will also make it more clear how to implement it in Python):

\begin{equation}
C=\frac{(\mathbf{u}-\overline{I})^{\top}(\mathbf{v}-\overline{J})}{\sqrt{(\mathbf{u}-\overline{I})^{\mathrm{T}}(\mathbf{u}-\overline{I})} \sqrt{(\mathbf{v}-\overline{J})^{\mathrm{T}}(\mathbf{v}-\overline{J})}}
\end{equation}

where $\mathbf{u}$ and $\mathbf{v}$ are vectors of the pixels intensities of the images $I$ and $J$, respectively:

\begin{equation}
u=\left[ \begin{array}{c}{I(1)} \\ {I(2)} \\ {\vdots} \\ {I(n)}\end{array}\right], v=\left[ \begin{array}{c}{J(1)} \\ {J(2)} \\ {\vdots} \\ {J(n)}\end{array}\right]
\end{equation}

### (A)
### EXERCISE:
The provided function `correlation()` in `# SECTION 3` of the `registration.py` module contains a template for implementing the normalized cross-correlation metric. Most of the functionality such as parameter checking and pre-processing of the images is already implemented. The only piece of code that is missing is the computation of the normalized cross-correlation using the equation above.

Implement the missing functionality in the `correlation()` function. Note that the mean intensity is already subtracted from the images.

### (B)
### EXERCISE:
Test your implementation using the template `correlation_test()` script provided in `# SECTION 3` of the `registration_tests.py` module. For example, you can make sure that `correlation(I,I)`, i.e. the normalized cross-correlation of any image with itself, returns 1. Use some other properties of normalized cross-correlation in order to further test your implementation. (HINT: How does a linear transformation of the intensities of the images affect the normalized cross-correlation coefficient?)

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
from registration_tests import correlation_test

correlation_test()

### (C)
### QUESTION:
Under which assumptions is the normalized cross-correlation the optimal similarity metric?

## 3.2. Joint histogram

The `joint_histogram()` function in `# SECTION 3` of the `registration.py` module contains an almost complete implementation of computation of the joint histogram of two images. We use the joint histogram as an estimate of the joint probability mass function (p.m.f) of two images. This function informs us of the probability that two intensities co-occur (appear together) at the same location in the two images.

### (A)
### EXERCISE:
Go over the implementation and make sure you understand the functionality of all the steps in the code. Implement the last missing step in the computation of the joint histogram.

### (B)
### QUESTION:
One of the parameters of the function is num bins, which defines the number of bins of the joint histogram. The default value for this parameter in this implementation is chosen to be 16. We mostly work with 8-bit images that have 256 possible values for the pixel intensities, which means that num bins can go as high as 256. However, there is a trade-off between choosing num bins too low or too high. What is this trade-off?

## 3.3. Mutual Information

Given the joint p.m.f. of the two images $p_{I, J}$ and the two marginal p.m.f.'s (the p.m.f.'s of the individual images) $p_{I}$ and $p_{I}$, the mutual information can be computed according to the following equation:

\begin{equation}
M I(I, J)=\sum_{i=1}^{n} \sum_{j=1}^{n} p_{I, J}(i, j) \log \frac{p_{I, J}(i, j)}{p_{I}(i) p_{J}(j)}
\end{equation}

Remember that the joint histogram is an estimate of the joint p.m.f. Thus, in the previous equation, we can "plug in" the joint histogram for $p_{I, J}$, and analogously, the marginal histograms (the histograms of the individual images) for $p_{I}$ and $p_{J}$. The two sum operators go over all bins in the joint histogram.

### (A)
### EXERCISE:
A template for implementation of mutual information given a joint histogram of two images is given in the Python function `mutual_information()` in `# SECTION 3` of the `registration.py` module. As before, the file already contains all the pre-processing steps but the actual computation of the mutual information is missing. The only missing piece of code in the template file is implementation of the above formula for mutual information. Implement the missing functionality.

### (B)
### EXERCISE:
Use some of the properties of mutual information to test your implementation. Write these tests in the provided `mutual_information_test()` script in `# SECTION 3` of the `registration_tests.py` module. (HINT: What would be the mutual information of two random noise images? You can generate random noise `uint8` images with `np.random.randint(255, size=(512, 512))`)

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
from registration_tests import mutual_information_test

mutual_information_test()

### (C)
An alternative formula for mutual information is:

$$
MI(I,J)=H(I)+H(J)-H(I, J)
$$

In the previous equation, $H(I)$ and $H(J)$ is the entropy of the images $I$ and $J$ and $H(I,J)$ is their joint entropy. 

### EXERCISE:
Find out the equation for the entropy and implement mutual information, using this formula, in the `mutual_information_e()` function in `# SECTION 3` of the `registration.py` module. Test your implementation with the provided `mutual_information_e_test()` script. Make sure that both implementations output equal values (very small differences are possible due to rounding errors).


In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
from registration_tests import mutual_information_e_test

mutual_information_e_test()

### (D)
### QUESTION:
When is mutual information preferable over sum of squared errors and normalized cross-correlation as an image similarity metric? Motivate your answer.


### (E)
### QUESTION:
The output of `mutual_information()` is described as "*mutual information in nat units*". What change in the code would you have to make to output the mutual information in bits? Does it make a difference which units you output when you use the mutual information metric in practice (for example, to perform image registration)?